In [ ]:
!python scripts/prepare_rirs.py

In [20]:
import pysofaconventions as sofa
import numpy as np
import os
import spatialscaper as ss

In [ ]:
NSCAPES = 20  # number of soundscapes to generate
FOREGROUND_DIR = "datasets/sound_event_datasets/FSD50K_FMA"  # directory with FSD50K foreground sound files
RIR_DIR = "datasets/rir_datasets"  # directory containing Room Impulse Response (RIR) files
ROOM = "soundspaces_shoebox" 
FORMAT = "foa"  # output format specifier: could be 'mic' or 'foa'
N_EVENTS_MEAN = 15  # mean number of foreground events in a soundscape
N_EVENTS_STD = 6  # standard deviation of the number of foreground events
DURATION = 60.0  # duration in seconds of each soundscape
SR = 24000  # SpatialScaper default sampling rate for the audio files
OUTPUT_DIR = "output"  # Directory to store the generated soundscapes
REF_DB = (-65)  # Reference decibel level for the background ambient noise. Try making this random too!

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, FORMAT), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels"), exist_ok=True)

def generate_soundscape(index):
    print(f"Generating soundscape: {index + 1}/{NSCAPES}") 
    track_name = f"fold1_room1_mix{index+1:03d}"
    ssc = ss.Scaper(
        DURATION,
        FOREGROUND_DIR,
        RIR_DIR,
        FORMAT,
        ROOM,
        max_event_overlap=2,
        speed_limit=2.0,
    )
    ssc.ref_db = REF_DB
    ssc.add_background()

    # Add a random number of foreground events, based on the specified mean and standard deviation.
    n_events = int(np.random.normal(N_EVENTS_MEAN, N_EVENTS_STD))
    n_events = n_events if n_events > 0 else 1
    print(f"Number of events to add: {n_events}")
    for i in range(n_events):
        print(f"Adding event {i + 1}/{n_events}") 
        ssc.add_event()  # randomly choosing and spatializing an FSD50K sound event

    audiofile = os.path.join(OUTPUT_DIR, FORMAT, track_name)
    labelfile = os.path.join(OUTPUT_DIR, "labels", track_name)
    ssc.generate(audiofile, labelfile)

for iscape in range(NSCAPES):
    generate_soundscape(iscape)